# From dataframe to geojson

This notebook transforms the dataframe outputted from the scrape (see Zocdoc scrape.ipynb) into a frame grouped by zip code with columns containing HTML text as a string to be displayed alongside and on the map.  

In [638]:
import pandas as pd
from collections import Counter

In [639]:
df = pd.read_csv('all-zocdoc-data-nyc.csv')

In [685]:
df.head()

,name,specialty,street_address,city,state,zip,rating,num_reviews,next_appt,num_appts_next_5days,sponsored,education,languages,gender,npi,insurance,name_address_string
0,"Dr. Igor Kletsman, MD",Primary Care Doctor,"23 W 45th Street, 2nd Floor",New York,NY,10036,4.70,3249,2021-12-14 08:30:00,3,False,['Medical School - Saba University School of M...,"['English', 'Russian', 'Polish', 'Ukrainian']",Male,1.447298e+09,"['1199SEIU', 'Aetna', 'Anthem Blue Cross', 'An...","<li>Dr. Igor Kletsman, MD (23 W 45th Street, 2..."
1,"Dr. Mukund Das, MD",Primary Care Doctor,"548 W 28th St, Ste 333",New York,NY,10001,4.43,21,2021-12-14 08:15:00,3,False,"[""Medical School - St. George's University Sch...",['English'],Male,1.831552e+09,"['Aetna', 'Caterpillar', 'Cigna', 'Cigna-Healt...","<li>Dr. Mukund Das, MD (548 W 28th St, Ste 333..."
2,"Dr. Weymin Hago, MD",Primary Care Doctor,"274 Madison Avenue, Room 1804",New York,NY,10016,4.93,1830,2021-12-28 13:30:00,0,False,"[""Medical School - Ross University School of M...","['English', 'Spanish']",Male,1.992941e+09,"['1199SEIU', 'AARP', 'Aetna', 'Anthem Blue Cro...","<li>Dr. Weymin Hago, MD (274 Madison Avenue, R..."
3,"Dr. Alexis Drullinsky, MD",Primary Care Doctor,"55 E 34th St, 1st Floor",New York,NY,10016,4.76,2391,2021-12-14 12:00:00,3,False,"['Medical School - Sackler School of Medicine,...","['English', 'Spanish']",Male,1.518930e+09,"['1199SEIU', 'Aetna', 'AgeWell New York', 'Ame...","<li>Dr. Alexis Drullinsky, MD (55 E 34th St, 1..."
4,"Dr. Jonathan Weinberger, MD",Primary Care Doctor,"145 E 32nd St, Suite 303",New York,NY,10016,4.92,628,2021-12-21 14:00:00,0,False,['Medical School - Albert Einstein College of ...,['English'],Male,1.851735e+09,"['1199SEIU', 'AARP', 'Aetna', 'Anthem Blue Cro...","<li>Dr. Jonathan Weinberger, MD (145 E 32nd St..."


## Generating the HTML for the map article text 

This section pulls from various columns in the dataframe to generate strings that will be displayed in the article alongside the map.

Creating a title for each page consisting of the zip and city

In [641]:
zip_city_grouping = df.groupby('zip').city.value_counts().groupby(level=0, group_keys=False).nlargest(1).reset_index(name='city_title')

In [642]:
zip_city_grouping.drop(labels='city_title', axis=1, inplace=True)

In [643]:
zip_city_grouping['title_html'] = '<h1>' + zip_city_grouping.zip.astype(str).str.zfill(5) + ' –⁠ ' + zip_city_grouping.city + '</h1>'


In [644]:
zip_city_grouping.drop(labels='city', axis=1, inplace=True)
zip_city_grouping.head()

,zip,title_html
0,7302,<h1>07302 –⁠ Jersey City</h1>
1,8536,<h1>08536 –⁠ Plainsboro Township</h1>
2,10001,<h1>10001 –⁠ New York</h1>
3,10002,<h1>10002 –⁠ New York</h1>
4,10003,<h1>10003 –⁠ New York</h1>


Creating an unordered list of doctos in each zip code:

In [645]:
df['name_address_string'] = '<li>' + df.name + ' (' + df.street_address + ')</li>' 

In [646]:
name_address_grouping = df.groupby('zip').name_address_string.sum().reset_index(name='doc_html')


In [647]:
name_address_grouping['doc_html'] = '<ul>' + name_address_grouping.doc_html + '</ul>'

In [648]:
name_address_grouping

,zip,doc_html
0,7302,"<ul><li>Dr. Deval Gadhvi, MD (34 Exchange Plac..."
1,8536,"<ul><li>Dr. Valerie Layne, DNP (3 Liberty St, ..."
2,10001,"<ul><li>Dr. Mukund Das, MD (548 W 28th St, Ste..."
3,10002,"<ul><li>Dr. Ameer Hmeidan, MD (104 Delancey St..."
4,10003,"<ul><li>Dr. John Adams, MD (119 5th Avenue, 2n..."
...,...,...
110,11426,"<ul><li>Dr. Steven Celestin, MD (24818 Union T..."
111,11428,"<ul><li>Dr. Charlene Andrews, MD (22001 Jamaic..."
112,11432,"<ul><li>Dr. Iffat Sadique, MD (168-06 Hillside..."
113,11434,"<ul><li>Candice Waith, PAC (16995 137th Ave)</..."


In [649]:
name_address_grouping.doc_html[0]

'<ul><li>Dr. Deval Gadhvi, MD (34 Exchange Place, Plaza II Harborside Financial Center)</li><li>Dr. Prahlad Gadhavi, MD (200 Hudson Street, Suite 145 Plaza 2)</li></ul>'

Getting a count of the total number of providers in each zip code

In [650]:
doc_count_grouping = df.groupby('zip').name.count().reset_index(name='doc_count')

In [651]:
#pandas.DataFrame.loc[condition, column_label] = new_value
doc_count_grouping['doc_count_html'] = '<p>There are ' + doc_count_grouping.doc_count.astype(str) + ' primary care providers in this zip code.</p>'
doc_count_grouping.loc[doc_count_grouping.doc_count == 1, 'doc_count_html'] = '<p>There is 1 primary care provider in this zip code.</p>'

In [652]:
doc_count_grouping.drop(labels='doc_count', axis=1, inplace=True)

Getting a count of the number of providers who speak Spanish and Chinese, the two most popular languages spoken in NYC after English

In [653]:
langs = df[["zip","languages"]]
langs['Spanish'] = langs.languages.str.contains('Spanish')
langs['Chinese'] = langs.languages.str.contains('Chinese')

/var/folders/tx/ryc062z51z1287rjpvrp4jlc0000gn/T/ipykernel_80762/3293109743.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  langs['Spanish'] = langs.languages.str.contains('Spanish')
/var/folders/tx/ryc062z51z1287rjpvrp4jlc0000gn/T/ipykernel_80762/3293109743.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  langs['Chinese'] = langs.languages.str.contains('Chinese')


In [654]:
spanish_counts = langs[langs.Spanish == True].groupby('zip').Spanish.count().reset_index(name='num_spanish')
chinese_counts = langs[langs.Chinese == True].groupby('zip').Chinese.count().reset_index(name='num_chinese')


In [655]:
spanish_counts['spanish_html'] = '<p>Spanish is spoken at ' + spanish_counts.num_spanish.astype(str) + " of the provider's offices</p>"
chinese_counts['chinese_html'] = '<p>Mandarin or Cantonese is spoken at ' + chinese_counts.num_chinese.astype(str) + " providers' offices</p>"


In [656]:
spanish_counts.drop(labels='num_spanish', axis=1, inplace=True)
chinese_counts.drop(labels='num_chinese', axis=1, inplace=True)

Getting a count of the total number of available appointments in the next five days that the providers have, combined

In [657]:
appt_counts_grouping = df.groupby('zip').num_appts_next_5days.sum().reset_index(name='total_appts')
appt_counts_grouping['total_appts_html'] = "<p>Among them, there are " + appt_counts_grouping.total_appts.astype(str) + " appointments available in the next 5 days.</p>"


Getting the soonest available appointment among all the providers in the zip code

In [658]:
df.next_appt = pd.to_datetime(df.next_appt)

In [659]:
next_appt_grouping = df.groupby(by=['zip','name']).next_appt.min().groupby(level=0, group_keys=False).nsmallest(1).reset_index()


In [660]:
next_appt_grouping['next_appt_html'] = "<p>The soonest available appointment is with " + next_appt_grouping.name + " on " + next_appt_grouping.next_appt.dt.strftime('%A, %B %-d at %-I:%M %p') + "</p>"

In [661]:
next_appt_grouping.drop(labels=['name', 'next_appt'], axis=1, inplace=True)

Joining all of the above dataframes. Each of these have a zip code column and a column containing a string with HTML, except from num_appt_grouping which retains the number of appointments (later used to determine color of each zip code on the map)

In [662]:
grouping_dfs = [zip_city_grouping, doc_count_grouping, name_address_grouping, spanish_counts, chinese_counts, appt_counts_grouping, next_appt_grouping]


In [686]:
article = pd.DataFrame(zip_city_grouping)
for frame in grouping_dfs[1:]:
    article = article.merge(frame, how='outer')
article.spanish_html = article.spanish_html.fillna('<p>None of the providers speak Spanish</p>')
article.chinese_html = article.chinese_html.fillna('<p>None of the providers speak Chinese</p>')
article.head()

,zip,title_html,doc_count_html,doc_html,spanish_html,chinese_html,total_appts,total_appts_html,next_appt_html
0,7302,<h1>07302 –⁠ Jersey City</h1>,<p>There are 2 primary care providers in this ...,"<ul><li>Dr. Deval Gadhvi, MD (34 Exchange Plac...",<p>Spanish is spoken at 2 of the provider's of...,<p>None of the providers speak Chinese</p>,6,"<p>Among them, there are 6 appointments availa...",<p>The soonest available appointment is with D...
1,8536,<h1>08536 –⁠ Plainsboro Township</h1>,<p>There is 1 primary care provider in this zi...,"<ul><li>Dr. Valerie Layne, DNP (3 Liberty St, ...",<p>None of the providers speak Spanish</p>,<p>None of the providers speak Chinese</p>,1,"<p>Among them, there are 1 appointments availa...",<p>The soonest available appointment is with D...
2,10001,<h1>10001 –⁠ New York</h1>,<p>There are 3 primary care providers in this ...,"<ul><li>Dr. Mukund Das, MD (548 W 28th St, Ste...",<p>None of the providers speak Spanish</p>,<p>None of the providers speak Chinese</p>,9,"<p>Among them, there are 9 appointments availa...",<p>The soonest available appointment is with D...
3,10002,<h1>10002 –⁠ New York</h1>,<p>There are 6 primary care providers in this ...,"<ul><li>Dr. Ameer Hmeidan, MD (104 Delancey St...",<p>None of the providers speak Spanish</p>,<p>Mandarin or Cantonese is spoken at 1 provid...,0,"<p>Among them, there are 0 appointments availa...",<p>The soonest available appointment is with D...
4,10003,<h1>10003 –⁠ New York</h1>,<p>There are 10 primary care providers in this...,"<ul><li>Dr. John Adams, MD (119 5th Avenue, 2n...",<p>Spanish is spoken at 1 of the provider's of...,<p>None of the providers speak Chinese</p>,12,"<p>Among them, there are 12 appointments avail...",<p>The soonest available appointment is with D...


Combining each of these HTML string columns into one big column and generating an output dataframe with the necessary info

In [664]:
article = article.fillna('')

In [665]:
article['properties.article'] = article.title_html + '<p></p>' + article.doc_count_html + article.doc_html + \
                                '<p></p>' + article.spanish_html + article.chinese_html + '<p></p>' + \
                                article.total_appts_html + '<p></p>' + article.next_appt_html

In [666]:
output = article[['zip', 'title_html', 'properties.article', 'total_appts']]

In [667]:
output = output.rename(columns={'title_html':'properties.headline'})

Adding color based on number of appointments in each zip code

In [668]:
color_ramp = ['#B8F1F5', '#A7E3F5', '#69AEDB', '#6CA7F5', '#6487E8']

In [669]:
output.total_appts.describe()

count    115.00000
mean       5.13913
std        7.06348
min        0.00000
25%        0.00000
50%        3.00000
75%        6.00000
max       35.00000
Name: total_appts, dtype: float64

In [670]:
output.loc[output.total_appts.between(0, 3, inclusive='both'), 'properties.color'] = color_ramp[0]
output.loc[output.total_appts.between(4, 6, inclusive='both'), 'properties.color'] = color_ramp[1]
output.loc[output.total_appts.between(7, 10, inclusive='both'), 'properties.color'] = color_ramp[2]
output.loc[output.total_appts.between(11, 20, inclusive='both'), 'properties.color'] = color_ramp[3]
output.loc[output.total_appts.between(21, 35, inclusive='both'), 'properties.color'] = color_ramp[4]

In [671]:
output

,zip,properties.headline,properties.article,total_appts,properties.color
0,7302,<h1>07302 –⁠ Jersey City</h1>,<h1>07302 –⁠ Jersey City</h1><p></p><p>There a...,6,#A7E3F5
1,8536,<h1>08536 –⁠ Plainsboro Township</h1>,<h1>08536 –⁠ Plainsboro Township</h1><p></p><p...,1,#B8F1F5
2,10001,<h1>10001 –⁠ New York</h1>,<h1>10001 –⁠ New York</h1><p></p><p>There are ...,9,#69AEDB
3,10002,<h1>10002 –⁠ New York</h1>,<h1>10002 –⁠ New York</h1><p></p><p>There are ...,0,#B8F1F5
4,10003,<h1>10003 –⁠ New York</h1>,<h1>10003 –⁠ New York</h1><p></p><p>There are ...,12,#6CA7F5
...,...,...,...,...,...
110,11426,<h1>11426 –⁠ Bellerose</h1>,<h1>11426 –⁠ Bellerose</h1><p></p><p>There are...,3,#B8F1F5
111,11428,<h1>11428 –⁠ Queens Village</h1>,<h1>11428 –⁠ Queens Village</h1><p></p><p>Ther...,0,#B8F1F5
112,11432,<h1>11432 –⁠ Jamaica</h1>,<h1>11432 –⁠ Jamaica</h1><p></p><p>There is 1 ...,3,#B8F1F5
113,11434,<h1>11434 –⁠ Jamaica</h1>,<h1>11434 –⁠ Jamaica</h1><p></p><p>There are 3...,1,#B8F1F5


## Creating a geojson based on this dataframe

Using an NYC geojson file downloaded from https://data.beta.nyc/dataset/nyc-zip-code-tabulation-areas/resource/894e9162-871c-4552-a09c-c6915d8783fb, I created a geojson file to use in Mapbox GL

In [672]:
import requests
import json
import numpy as np
import pandas as pd
from pandas import json_normalize


In [673]:
with open('zip_code_040114.geojson') as json_data:
    geometry_data = json.load(json_data)

In [674]:
map_df = pd.DataFrame.from_dict(json_normalize(geometry_data['features']), orient='columns')

In [675]:
map_df.head()

,type,properties.ZIPCODE,properties.BLDGZIP,properties.PO_NAME,properties.POPULATION,properties.AREA,properties.STATE,properties.COUNTY,properties.ST_FIPS,properties.CTY_FIPS,properties.URL,properties.SHAPE_AREA,properties.SHAPE_LEN,geometry.type,geometry.coordinates
0,Feature,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,Polygon,"[[[-73.80584847647393, 40.682909325509684], [-..."
1,Feature,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,Polygon,"[[[-73.9373976313981, 40.67972958831804], [-73..."
2,Feature,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,Polygon,"[[[-73.90294132545436, 40.67083977496735], [-7..."
3,Feature,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,Polygon,"[[[-73.9579731604348, 40.670656958042926], [-7..."
4,Feature,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,Polygon,"[[[-73.97208109564255, 40.650596586343454], [-..."


In [676]:
map_df.drop(labels=['properties.URL', 'properties.SHAPE_AREA', 'properties.SHAPE_LEN', 'properties.CTY_FIPS', 'properties.ST_FIPS', 'properties.BLDGZIP', 'properties.STATE'], axis=1, inplace=True)

In [677]:
map_df.head()

,type,properties.ZIPCODE,properties.PO_NAME,properties.POPULATION,properties.AREA,properties.COUNTY,geometry.type,geometry.coordinates
0,Feature,11436,Jamaica,18681.0,2.269930e+07,Queens,Polygon,"[[[-73.80584847647393, 40.682909325509684], [-..."
1,Feature,11213,Brooklyn,62426.0,2.963100e+07,Kings,Polygon,"[[[-73.9373976313981, 40.67972958831804], [-73..."
2,Feature,11212,Brooklyn,83866.0,4.197210e+07,Kings,Polygon,"[[[-73.90294132545436, 40.67083977496735], [-7..."
3,Feature,11225,Brooklyn,56527.0,2.369863e+07,Kings,Polygon,"[[[-73.9579731604348, 40.670656958042926], [-7..."
4,Feature,11218,Brooklyn,72280.0,3.686880e+07,Kings,Polygon,"[[[-73.97208109564255, 40.650596586343454], [-..."


In [678]:
output = output.rename(columns={'zip':'properties.ZIPCODE'})
output.drop(labels='total_appts', axis=1, inplace=True)
output['properties.ZIPCODE'] = output['properties.ZIPCODE'].astype(str)

In [679]:
map_df = map_df.merge(output, on='properties.ZIPCODE')

In [680]:
map_df = map_df.rename(columns={'properties.ZIPCODE':'properties.name'})

In [681]:
ok_json = json.loads(map_df.to_json(orient='records'))

In [682]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data

In [683]:
geo_format = process_to_geojson(ok_json)

In [684]:
with open('geo-data.js', 'w') as outfile:
    outfile.write("var infoData = ")
with open('geo-data.js', 'a') as outfile:
    json.dump(geo_format, outfile)